<a href="https://colab.research.google.com/github/gongbububu/gongbububu/blob/main/3CVcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def find_zero_crossings_and_interpolate(data_str):
    # 데이터를 튜플 리스트로 변환 (공백을 기준으로 두 숫자씩 묶기)
    data = [tuple(map(float, data_str[i:i+2])) for i in range(0, len(data_str), 2)]

    # 음수로 바뀌는 구간과 양수로 바뀌는 구간을 찾기
    crossing_points = []

    for i in range(1, len(data)):
        x_prev, y_prev = data[i-1]
        x_curr, y_curr = data[i]

        if x_prev > 0 and x_curr <= 0:
            # x값이 양수에서 음수로 바뀌는 구간을 발견
            crossing_points.append(("negative_cross", data[i-1], data[i]))
        elif x_prev < 0 and x_curr >= 0:
            # x값이 음수에서 양수로 바뀌는 구간을 발견
            crossing_points.append(("positive_cross", data[i-1], data[i]))

    if len(crossing_points) < 2:
        raise ValueError("두 개의 음수-양수 교차점을 찾을 수 없습니다.")

    # 음수로 바뀌는 구간에서 선형 보간법으로 y값을 구함
    negative_cross = crossing_points[0]
    x1, y1 = negative_cross[1]
    x2, y2 = negative_cross[2]

    # 선형 보간법: y = m * x + b (m은 기울기, b는 절편)
    m_neg = (y2 - y1) / (x2 - x1)
    b_neg = y1 - m_neg * x1
    y_at_zero_neg = m_neg * 0 + b_neg  # x = 0일 때의 y값

    # 양수로 바뀌는 구간에서 선형 보간법으로 y값을 구함
    positive_cross = crossing_points[1]
    x3, y3 = positive_cross[1]
    x4, y4 = positive_cross[2]

    m_pos = (y4 - y3) / (x4 - x3)
    b_pos = y3 - m_pos * x3
    y_at_zero_pos = m_pos * 0 + b_pos  # x = 0일 때의 y값

    # 두 y값의 차이
    result = (abs(y_at_zero_pos - y_at_zero_neg)) / 2

    return (x1, y1), (x2, y2), (x3, y3), (x4, y4), result

# 사용자로부터 데이터를 입력받기
data_str = input("데이터를 입력하세요 (공백으로 구분된 숫자들): ").split()

# 함수 호출
try:
    point1, point2, point3, point4, result = find_zero_crossings_and_interpolate(data_str)
    print(f"첫 번째 음수 교차점 (x1, y1): {point1}")
    print(f"두 번째 음수 교차점 (x2, y2): {point2}")
    print(f"첫 번째 양수 교차점 (x3, y3): {point3}")
    print(f"두 번째 양수 교차점 (x4, y4): {point4}")
    print(f"최종 결과: {result}")
except ValueError as e:
    print(e)


데이터를 입력하세요 (공백으로 구분된 숫자들): 0.051653475	0.047018057 0.050723165	0.043330871 0.049699813	0.040445921 0.048676431	0.037996026 0.047684103	0.03572509 0.046660751	0.033623859 0.045699418	0.031778723 0.044676036	0.029949017 0.043683708	0.028205703 0.042722374	0.02651793 0.041667998	0.02491655 0.04067564	0.023386136 0.039683312	0.021957545 0.03865993	0.0204765 0.037636578	0.019094192 0.036675245	0.017770508 0.035651892	0.016428312 0.034659535	0.015200279 0.033667207	0.013950647 0.032705873	0.012768897 0.031651497	0.011528522 0.030659139	0.010389968 0.029635787	0.009214389 0.028116256	0.007562033 0.027123898	0.006485183 0.026069552	0.005462022 0.025108188	0.004299086 0.024084836	0.003263273 0.023123503	0.002241963 0.022131145	0.001253051 0.018068731	-0.002742401 0.017076403	-0.003757232 0.016115069	-0.0046363 0.015091687	-0.005600219 0.014068335	-0.006564138 0.013075978	-0.007428704 0.01208365	-0.008370407 0.011060297	-0.009262434 0.009881884	-0.010350828 0.00892055	-0.011153061 0.007897168	-0

In [ ]:
import itertools
import numpy as np

def calculate_slope_and_intercept(x_values, y_values):
    # 선형 회귀를 통해 기울기와 절편을 구하는 함수
    A = np.vstack([x_values, np.ones(len(x_values))]).T
    m, c = np.linalg.lstsq(A, y_values, rcond=None)[0]
    return m, c

def find_possible_slopes_and_points(x_values, y_values):
    possible_slopes = []

    # 각 x에 대해 하나의 y값을 선택하는 조합을 만들어 기울기를 계산
    for selected_y_values in itertools.product(*y_values):
        # 선형 회귀를 통해 기울기 계산
        slope, intercept = calculate_slope_and_intercept(x_values, selected_y_values)
        possible_slopes.append((slope, selected_y_values))

    # 기울기를 내림차순으로 정렬
    possible_slopes = sorted(possible_slopes, key=lambda x: x[0], reverse=True)
    return possible_slopes

# 사용자로부터 데이터 입력 받기
x_values = [0.0002, 0.0005, 0.001, 0.002, 0.003]
y_values = []

# 각 x 값에 대해 y값 3개씩 입력 받기
for x in x_values:
    y_input = input(f"X={x}일 때 가능한 y의 좌표 3개를 입력하세요 (공백으로 구분): ").split()
    y_values.append(list(map(float, y_input)))

# 가능한 모든 기울기 계산
all_slopes = find_possible_slopes_and_points(x_values, y_values)

# 결과 출력
print("\n가능한 기울기들 (내림차순):")
for slope, selected_y_values in all_slopes:
    print(f"기울기: {slope:.6f}, 선택된 y값: {selected_y_values}")


X=0.0002일 때 가능한 y의 좌표 3개를 입력하세요 (공백으로 구분): 0.00922987 0.008904962 0.00880555
X=0.0005일 때 가능한 y의 좌표 3개를 입력하세요 (공백으로 구분): 0.011730243 0.01101535 0.010924149
X=0.001일 때 가능한 y의 좌표 3개를 입력하세요 (공백으로 구분): 0.013702388 0.013027908 0.012975584
X=0.002일 때 가능한 y의 좌표 3개를 입력하세요 (공백으로 구분): 0.016109996 0.015441625 0.015410362
X=0.003일 때 가능한 y의 좌표 3개를 입력하세요 (공백으로 구분): 0.017588872 0.01691899 0.016903597

가능한 기울기들 (내림차순):
기울기: 3.050417, 선택된 y값: (0.00880555, 0.010924149, 0.012975584, 0.016109996, 0.017588872)
기울기: 3.047068, 선택된 y값: (0.00880555, 0.010924149, 0.013027908, 0.016109996, 0.017588872)
기울기: 3.035995, 선택된 y값: (0.00880555, 0.01101535, 0.012975584, 0.016109996, 0.017588872)
기울기: 3.032646, 선택된 y값: (0.00880555, 0.01101535, 0.013027908, 0.016109996, 0.017588872)
기울기: 3.029082, 선택된 y값: (0.008904962, 0.010924149, 0.012975584, 0.016109996, 0.017588872)
기울기: 3.025733, 선택된 y값: (0.008904962, 0.010924149, 0.013027908, 0.016109996, 0.017588872)
기울기: 3.014660, 선택된 y값: (0.008904962, 0.01101535, 0.012975584, 0.01

In [ ]:
!pip install pyxlsb

In [3]:
# 이게 찐 코드


import os
import numpy as np
import itertools
import pandas as pd

def find_zero_crossings_and_interpolate(df):
    data = df[['v-ocv', 'mA']].values
    crossing_points = []

    for i in range(1, len(data)):
        x_prev, y_prev = data[i-1]
        x_curr, y_curr = data[i]

        if x_prev > 0 and x_curr <= 0:
            crossing_points.append(("negative_cross", (x_prev, y_prev), (x_curr, y_curr)))
        elif x_prev < 0 and x_curr >= 0:
            crossing_points.append(("positive_cross", (x_prev, y_prev), (x_curr, y_curr)))

    if len(crossing_points) < 2:
        return None

    neg_cross = crossing_points[0]
    (x1, y1), (x2, y2) = neg_cross[1], neg_cross[2]
    m_neg = (y2 - y1) / (x2 - x1)
    b_neg = y1 - m_neg * x1
    y_at_zero_neg = m_neg * 0 + b_neg

    pos_cross = crossing_points[1]
    (x3, y3), (x4, y4) = pos_cross[1], pos_cross[2]
    m_pos = (y4 - y3) / (x4 - x3)
    b_pos = y3 - m_pos * x3
    y_at_zero_pos = m_pos * 0 + b_pos

    result = abs(y_at_zero_pos - y_at_zero_neg) / 2
    return result

def update_excel_with_dataframe(file_path):
    df = pd.read_excel(file_path, sheet_name="DCData1")

    df.drop(columns=['ListNo', 'TestTime_s', 'CycleTime_s', 'StepNo',
                     'StepTime_s', 'SumQ_Ah', 'AbsQ_Ah', 'Power_W', 'Load_Ohm',
                     'Temperature_C', 'Auxiliary1_V', 'Auxiliary2_V', 'Auxiliary3_V', 'RangeInfo'],
            inplace=True)

    df['v-ocv'] = df['Voltage_V'] - df['OCP_V']
    df['mA'] = df['_Current_A'] * 1000

    df.dropna(subset=['v-ocv', 'mA'], inplace=True)

    # 🔹 **전처리된 데이터 저장**
    processed_file = f"{os.path.splitext(file_path)[0]}_processed.xlsx"
    df.to_excel(processed_file, sheet_name="Processed Data", index=False)
    print(f"✅ 전처리된 데이터 저장 완료: {processed_file}")

    file_numbers = [1, 2, 3, 4, 5]
    cycle_numbers = [0, 1, 2, 3]

    results = {}

    for file_no in file_numbers:
        for cycle_no in cycle_numbers:
            filtered_df = df[(df['FileNo'] == file_no) & (df['CycleNo'] == cycle_no)].copy()
            if not filtered_df.empty:
                result = find_zero_crossings_and_interpolate(filtered_df)
                if result is not None:
                    results[(file_no, cycle_no)] = result

    return results

def calculate_slope_and_intercept(x_values, y_values):
    A = np.vstack([x_values, np.ones(len(x_values))]).T
    m, c = np.linalg.lstsq(A, y_values, rcond=None)[0]
    return m, c

def find_possible_slopes_and_points(x_values, y_values):
    possible_slopes = []
    for selected_y_values in itertools.product(*y_values):
        slope, intercept = calculate_slope_and_intercept(x_values, selected_y_values)
        possible_slopes.append((slope, selected_y_values))

    possible_slopes = sorted(possible_slopes, key=lambda x: x[0], reverse=True)
    return possible_slopes

# 🔹 **원본 파일명에서 '_result' 추가한 파일명 생성**
file_path = "/content/CB_450_3CV_01 (1).xlsx"
file_name, file_ext = os.path.splitext(file_path)  # 확장자 분리
output_file = f"{file_name}_result{file_ext}"  # '_result' 추가한 파일명 생성

# 🔹 **데이터 처리**
results = update_excel_with_dataframe(file_path)

# 🔹 **최종 결과 DataFrame 생성**
final_results_df = pd.DataFrame(
    [(file_no, cycle_no, value) for (file_no, cycle_no), value in results.items()],
    columns=["FileNo", "CycleNo", "최종 결과"]
)

# 🔹 **FileNo를 x_values로 변환**
file_no_to_x = {1: 0.0002, 2: 0.0005, 3: 0.001, 4: 0.002, 5: 0.003}
x_values = []
y_values = []

for file_no in range(1, 6):
    cycle_results = [results[(file_no, cycle_no)] for cycle_no in range(4) if (file_no, cycle_no) in results]
    if cycle_results:
        x_values.append(file_no_to_x[file_no])
        y_values.append(cycle_results)

# 🔹 **가능한 모든 기울기 계산**
all_slopes = find_possible_slopes_and_points(x_values, y_values)

# 🔹 **기울기 결과 DataFrame 생성 (열 별로 저장)**
max_y_values_len = max(len(y) for _, y in all_slopes)  # 최대 y 개수 확인
columns = ["기울기"] + [f"y{i+1}" for i in range(max_y_values_len)]  # 열 이름 설정

slope_results_data = []
for slope, selected_y_values in all_slopes:
    row = [slope] + list(selected_y_values)  # 각 기울기와 y 값들을 행으로 추가
    slope_results_data.append(row)

slope_results_df = pd.DataFrame(slope_results_data, columns=columns)

# 🔹 **엑셀 파일로 저장**
with pd.ExcelWriter(output_file) as writer:
    final_results_df.to_excel(writer, sheet_name="최종 결과", index=False)
    slope_results_df.to_excel(writer, sheet_name="가능한 기울기들", index=False)

print(f"✅ 엑셀 파일 저장 완료: {output_file}")


✅ 전처리된 데이터 저장 완료: /content/CB_450_3CV_01 (1)_processed.xlsx
✅ 엑셀 파일 저장 완료: /content/CB_450_3CV_01 (1)_result.xlsx


In [ ]:
import pandas as pd
import numpy as np
import itertools
import os

def update_excel_with_dataframe(file_path):
    # 엑셀 파일을 pandas DataFrame으로 읽기
    df = pd.read_excel(file_path, sheet_name="DCData_1")

    # 지정된 열 삭제
    df.drop(columns=['ListNo', 'TestTime_s', 'CycleTime_s', 'StepNo',
                     'StepTime_s', 'SumQ_Ah', 'AbsQ_Ah', 'Power_W', 'Load_Ohm','Temperature_C',
                     'Auxiliary1_V','Auxiliary2_V','Auxiliary3_V','RangeInfo'], inplace=True)

    # 새로운 열 추가
    df['v-ocv'] = df['Voltage_V'] - df['OCP_V']
    df['mA'] = df['_Current_A'] * 1000

    # 처리된 데이터 저장
    processed_file_path = file_path.replace(".xlsx", "_processed.xlsx")
    df.to_excel(processed_file_path, index=False)

    # 최종 결과 저장할 딕셔너리
    results = {}

    for file_no in range(1, 6):
        for cycle_no in range(0, 4):
            filtered_df = df[(df['FileNo'] == file_no) & (df['CycleNo'] == cycle_no)]
            if filtered_df.empty:
                continue

            data = list(zip(filtered_df['v-ocv'], filtered_df['mA']))
            crossing_points = []

            for i in range(1, len(data)):
                x_prev, y_prev = data[i-1]
                x_curr, y_curr = data[i]
                if x_prev > 0 and x_curr <= 0:
                    crossing_points.append((data[i-1], data[i]))
                elif x_prev < 0 and x_curr >= 0:
                    crossing_points.append((data[i-1], data[i]))

            if len(crossing_points) < 2:
                continue

            def linear_interpolation(x1, y1, x2, y2):
                m = (y2 - y1) / (x2 - x1)
                b = y1 - m * x1
                return m * 0 + b

            y_at_zero_neg = linear_interpolation(*crossing_points[0][0], *crossing_points[0][1])
            y_at_zero_pos = linear_interpolation(*crossing_points[1][0], *crossing_points[1][1])

            result = abs(y_at_zero_pos - y_at_zero_neg) / 2
            results[(file_no, cycle_no)] = result

    x_values = [0.0002, 0.0005, 0.001, 0.002, 0.003]
    y_values = [[] for _ in x_values]

    for (file_no, cycle_no), value in results.items():
        if file_no <= 5:
            y_values[file_no - 1].append(value)

    for i in range(len(y_values)):
        while len(y_values[i]) < 3:
            y_values[i].append(np.nan)

    def calculate_slope_and_intercept(x_values, y_values):
        A = np.vstack([x_values, np.ones(len(x_values))]).T
        m, c = np.linalg.lstsq(A, y_values, rcond=None)[0]
        return m, c

    all_slopes = []
    for selected_y_values in itertools.product(*y_values):
        slope, intercept = calculate_slope_and_intercept(x_values, selected_y_values)
        all_slopes.append((slope, selected_y_values))

    all_slopes = sorted(all_slopes, key=lambda x: x[0], reverse=True)

    output_file_path = file_path.replace(".xlsx", "_result.xlsx")
    with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
        result_df = pd.DataFrame(results.items(), columns=['FileNo_CycleNo', '최종 결과'])
        result_df[['FileNo', 'CycleNo']] = pd.DataFrame(result_df['FileNo_CycleNo'].tolist(), index=result_df.index)
        result_df = result_df[['FileNo', 'CycleNo', '최종 결과']]
        result_df.to_excel(writer, sheet_name='최종 결과', index=False)

        slope_data = {"기울기": [s[0] for s in all_slopes]}
        for i in range(5):
            slope_data[f"y{i+1}"] = [s[1][i] for s in all_slopes]
        slope_df = pd.DataFrame.from_dict(slope_data, orient='columns').T
        slope_df.to_excel(writer, sheet_name='가능한 기울기들', header=False)

    return output_file_path

file_path = "/content/241230_M430_01.xlsx"
update_excel_with_dataframe(file_path)


'/content/241230_M430_01_result.xlsx'

In [ ]:
!pip install xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 7.0 MB/s eta 0:00:00
